In [1]:
import nbformat
# notebook_filename='./procs-postgres-query.ipynb'
notebook_filename='./procs-postgres-blob.ipynb'
nb = nbformat.read(notebook_filename, as_version=4)

In [2]:
nb.metadata.kernelspec.name

'python3'

In [3]:
tag='parameters'
for idx, cell in enumerate(nb.cells):
    if 'tags' in cell.metadata and tag in cell.metadata.tags:
        print(tag)

In [4]:
for idx, cell in enumerate(nb.cells):
    print(idx, '-'*25)
    print(cell.cell_type, cell.metadata)
    print(cell.source)

0 -------------------------
code {'ExecuteTime': {'end_time': '2020-07-15T08:28:19.578365Z', 'start_time': '2020-07-15T08:28:19.184031Z'}}
import psycopg2
from sagas.profile.connections import config

def write_blob(part_id, path_to_file:str, file_extension):
    """ insert a BLOB into a table """
    conn = None
    try:
        # read data from a picture
        drawing = open(path_to_file, 'rb').read()
        # read database configuration
        params = config()
        # connect to the PostgresQL database
        conn = psycopg2.connect(**params)
        # create a new cursor object
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute("INSERT INTO part_drawings(part_id,file_extension,drawing_data) " +
                    "VALUES(%s,%s,%s)",
                    (part_id, file_extension, psycopg2.Binary(drawing)))
        # commit the changes to the database
        conn.commit()
        # close the communication with the PostgresQL database
     

In [5]:
code=''
snips=[]
for idx, cell in enumerate(nb.cells):
    if cell.cell_type=='code':
        snips.append(cell.source)

code='\n'.join(snips)
print(code)

import psycopg2
from sagas.profile.connections import config

def write_blob(part_id, path_to_file:str, file_extension):
    """ insert a BLOB into a table """
    conn = None
    try:
        # read data from a picture
        drawing = open(path_to_file, 'rb').read()
        # read database configuration
        params = config()
        # connect to the PostgresQL database
        conn = psycopg2.connect(**params)
        # create a new cursor object
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute("INSERT INTO part_drawings(part_id,file_extension,drawing_data) " +
                    "VALUES(%s,%s,%s)",
                    (part_id, file_extension, psycopg2.Binary(drawing)))
        # commit the changes to the database
        conn.commit()
        # close the communication with the PostgresQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
    

In [6]:
from source_indexer.anal_sources import tree_vis, AstNode
import ast, gast
# code = open(file).read()
tree = ast.parse(code)
gtree = gast.ast_to_gast(tree)

In [7]:
tree_vis(gtree)

Module: _
├── Import: _
├── ImportFrom: _
├── FunctionDef: write_blob
│   ├── Expr: _
│   ├── Assign: _
│   └── Try: _
│       ├── Assign: _
│       ├── Assign: _
│       ├── Assign: _
│       ├── Assign: _
│       ├── Expr: _
│       ├── Expr: _
│       └── Expr: _
├── FunctionDef: read_blob
│   ├── Expr: _
│   ├── Assign: _
│   └── Try: _
│       ├── Assign: _
│       ├── Assign: _
│       ├── Assign: _
│       ├── Expr: _
│       ├── Assign: _
│       ├── Expr: _
│       └── Expr: _
├── Expr: _
├── Expr: _
└── Expr: _


In [8]:
root=AstNode(ast_node=gtree)
root.functions

(write_blob, read_blob)

In [9]:
root.functions[0].ast_node.args._fields

('args', 'vararg', 'kwonlyargs', 'kw_defaults', 'kwarg', 'defaults')

In [10]:
f=root.functions[0].ast_node
code.split('\n')[f.lineno-1:f.lineno+1]

['def write_blob(part_id, path_to_file:str, file_extension):',
 '    """ insert a BLOB into a table """']

In [11]:
def get_args(args):
    return [f"{a.id} {a.annotation.id if a.annotation else '_'}" for a in args]
fns=[(f.ast_node.name, f.ast_node.lineno, f.doc_str, get_args(f.ast_node.args.args)) 
     for f in root.functions]
fns

[('write_blob',
  4,
  'insert a BLOB into a table ',
  ['part_id _', 'path_to_file str', 'file_extension _']),
 ('read_blob',
  30,
  'read BLOB data from a table ',
  ['part_id _', 'path_to_dir _'])]

In [12]:
import marisa_trie
import msgpack
keys=[r[0] for r in fns]
values=[msgpack.packb(r, use_bin_type=True) for r in fns]
trie = marisa_trie.BytesTrie(zip(keys, values))
'write_blob' in trie, trie.keys('write')

(True, ['write_blob'])

In [13]:
val=trie['write_blob'][0]
msgpack.unpackb(val, raw=False)

['write_blob',
 4,
 'insert a BLOB into a table ',
 ['part_id _', 'path_to_file str', 'file_extension _']]

In [14]:
trie.prefixes('read_blobx')

['read_blob']

In [15]:
trie.save('blob/functions.marisa')

In [16]:
trie2 = marisa_trie.BytesTrie()
trie2.load('blob/functions.marisa')
val=trie2['write_blob'][0]
msgpack.unpackb(val, raw=False)

['write_blob',
 4,
 'insert a BLOB into a table ',
 ['part_id _', 'path_to_file str', 'file_extension _']]

In [17]:
from whoosh.index import create_in,open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser

from jieba.analyse.analyzer import ChineseAnalyzer

analyzer = ChineseAnalyzer()

# fns=[(f.ast_node.name, f.ast_node.lineno, f.doc_str, get_args(f.ast_node.args.args)) 
#      for f in root.functions]
# notebook_filename
schema = Schema(name=TEXT(stored=True), 
                path=ID(stored=True), 
                comment=TEXT(stored=True, analyzer=analyzer))

In [18]:
import os
if not os.path.exists("indexes"):
    os.mkdir("indexes")

ix = create_in("indexes", schema) # for create new index
#ix = open_dir("indexes") # for read only
writer = ix.writer()

In [19]:
for f in root.functions:
    writer.add_document(
        name=f.ast_node.name,
        path=notebook_filename,
        comment=f.doc_str,
    )
writer.commit()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/jieba.cache
Loading model cost 0.849 seconds.
Prefix dict has been built successfully.


In [20]:
searcher = ix.searcher()
parser = QueryParser("comment", schema=ix.schema)

In [21]:
keyword='blob'
q = parser.parse(keyword)
results = searcher.search(q)
for hit in results:
    print(hit.highlights("comment"))

insert a <b class="match term0">BLOB</b> into a table
read <b class="match term0">BLOB</b> data from a table


In [22]:
[t.text for t in analyzer("我的好朋友是李明;我爱北京天安门;IBM和Microsoft; I have a dream. this is intetesting and interested me a lot")]

['我',
 '好',
 '朋友',
 '是',
 '李明',
 '我',
 '爱',
 '北京',
 '天安',
 '天安门',
 'ibm',
 'microsoft',
 'dream',
 'intetest',
 'interest',
 'me',
 'lot']